In [71]:
import ast
import fastapi
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
app = fastapi.FastAPI()

steam_games = pd.read_csv("data/steam_games.csv", index_col=0, parse_dates=["date"])
steam_games["genres"] = steam_games["genres"].fillna("[]")
steam_games['genres'] = steam_games['genres'].apply(eval)

users_items = pq.read_table('data/users_items.parquet')
users_items = users_items.to_pandas()
users_reviews = pq.read_table('data/users_reviews.parquet')
users_reviews = users_reviews.to_pandas()

model_data = pd.read_csv("data/model_data.csv", index_col=0, parse_dates=["date"])
model_data['popular_genres'].fillna('', inplace=True)
model_data['common_genres'].fillna('', inplace=True)
model_data['unpopular_genres'].fillna('', inplace=True)

@app.get("/")
def test():
    return {"API STEAM GAMES ACTIVE"}

In [19]:
@app.get("/developer/{developer}")
def developer(developer: str):

    developer_data = steam_games[steam_games["developer"] == developer]

    game_count_by_year = developer_data["date"].dt.year.value_counts()
    game_count_by_year_df = pd.DataFrame(game_count_by_year).reset_index()
    game_count_by_year_df.columns = ["year", "item_count"]

    free_game_count_by_year = developer_data["date"][developer_data["price"] == 0].dt.year.value_counts()
    free_game_count_by_year_df = pd.DataFrame(free_game_count_by_year).reset_index()
    free_game_count_by_year_df.columns = ["year", "free_item_count"]

    game_count_by_year_df = pd.merge(game_count_by_year_df, free_game_count_by_year_df, on="year", how="left")    
    game_count_by_year_df["free_content"] = game_count_by_year_df["free_item_count"] * 100 / game_count_by_year_df["item_count"]
    game_count_by_year_df.drop(columns="free_item_count", inplace=True)

    result_dict = game_count_by_year_df.to_dict(orient="records")
    return result_dict

developer("Digital Leisure Inc.")

[{'year': 1983, 'item_count': 1, 'free_content': nan},
 {'year': 1984, 'item_count': 1, 'free_content': nan},
 {'year': 1991, 'item_count': 1, 'free_content': nan},
 {'year': 2014, 'item_count': 1, 'free_content': 100.0},
 {'year': 2015, 'item_count': 1, 'free_content': 100.0}]

In [46]:
@app.get("/userdata/{user_id}")
def userdata(user_id: str):

    user_items = users_items[users_items["user_id"] == user_id ]
    user_reviews = users_reviews[users_reviews["user_id"] == user_id]

    user_items = user_items.merge(steam_games[["game_id", "price"]], on="game_id", how="left")    
    spent_money = user_items["price"].sum()

    item_count = user_reviews.shape[0]
    recommend_rate = user_reviews["recommend"].value_counts()[True] * 100 / item_count
    
    user_data = {"user_id": user_id, "spent_money": spent_money, "recommend_rate": recommend_rate, "item_count": item_count}

    return user_data

userdata("76561197970982479")

{'user_id': '76561197970982479',
 'spent_money': 3424.31,
 'recommend_rate': 100.0,
 'item_count': 3}

In [185]:
@app.get("/userforgenre/{genre}")
def UserForGenre(genre: str):

    genre_items = users_items.merge(steam_games[["genres", "game_id", "date"]], on="game_id", how="left")
    genre_items["genres"] = genre_items["genres"].fillna("[]")
    genre_items = genre_items[genre_items["genres"].apply(lambda x: genre in x)]

    data = genre_items.groupby("user_id")["playtime_forever"].sum()
    df = pd.DataFrame(data).reset_index().sort_values(by="playtime_forever", ascending=False)
    most_hours_player = df.iloc[0][["user_id"]][0]

    hours_by_year = genre_items.groupby(genre_items["date"].dt.year)["playtime_forever"].sum()
    hours_by_year = [{"Año": year, "Horas": hours} for year, hours in hours_by_year.items()]

    return {"Player with most hours played for " + genre: most_hours_player,
        "Horas jugadas": hours_by_year}

UserForGenre("Action")

{'Player with most hours played for Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1983.0, 'Horas': 3582},
  {'Año': 1984.0, 'Horas': 384},
  {'Año': 1988.0, 'Horas': 30241},
  {'Año': 1989.0, 'Horas': 607},
  {'Año': 1990.0, 'Horas': 18787},
  {'Año': 1991.0, 'Horas': 2502},
  {'Año': 1992.0, 'Horas': 1925},
  {'Año': 1993.0, 'Horas': 211807},
  {'Año': 1994.0, 'Horas': 121057},
  {'Año': 1995.0, 'Horas': 222132},
  {'Año': 1996.0, 'Horas': 70061},
  {'Año': 1997.0, 'Horas': 687668},
  {'Año': 1998.0, 'Horas': 3089111},
  {'Año': 1999.0, 'Horas': 3040470},
  {'Año': 2000.0, 'Horas': 18663625},
  {'Año': 2001.0, 'Horas': 1480065},
  {'Año': 2002.0, 'Horas': 2680111},
  {'Año': 2003.0, 'Horas': 15075311},
  {'Año': 2004.0, 'Horas': 134255188},
  {'Año': 2005.0, 'Horas': 15670732},
  {'Año': 2006.0, 'Horas': 471120762},
  {'Año': 2007.0, 'Horas': 24553464},
  {'Año': 2008.0, 'Horas': 50035377},
  {'Año': 2009.0, 'Horas': 217871749},
  {'Año': 2010.0, 'Horas': 219922810},
  {'Año': 2011.0

In [171]:
@app.get("/best_developer_year/{year}")
def best_developer_year(year: int):

    developer_reviews = users_reviews.merge(steam_games[["game_id", "developer"]], on="game_id", how="left")
    developer_reviews = developer_reviews[developer_reviews["date"].dt.year == year ]

    positive_sentiment_items = developer_reviews[developer_reviews["sentiment"] == 2 ]
    
    positive_sentiment_items = positive_sentiment_items.groupby("developer")["sentiment"].count()
    positive_sentiment_items = pd.DataFrame(positive_sentiment_items).reset_index().sort_values(by="sentiment", ascending=False)

    recommended_items = developer_reviews[developer_reviews["recommend"] == True ]
    recommended_items = recommended_items.groupby("developer")["recommend"].count()
    recommended_items = pd.DataFrame(recommended_items).reset_index().sort_values(by="recommend", ascending=False)
    
    top_developers = pd.merge(positive_sentiment_items, recommended_items, on="developer")
    top_developers["recomendations"] = top_developers["sentiment"] + top_developers["recommend"]

    return [{"Position 1:": top_developers.iloc[0]["developer"]},
            {"Position 2:": top_developers.iloc[1]["developer"]},
            {"Position 3:": top_developers.iloc[2]["developer"]}]
    
best_developer_year(2010)

[{'Position 1:': 'Valve'},
 {'Position 2:': 'Tripwire Interactive'},
 {'Position 3:': 'DONTNOD Entertainment,Feral Interactive (Mac),Feral Interactive (Linux)'}]

In [129]:
@app.get("/developer_reviews_analysis/{developer}")
def developer_reviews_analysis(developer: str):

    developer_reviews = users_reviews.merge(steam_games[["game_id", "developer"]], on="game_id", how="left")
    developer_reviews = developer_reviews[developer_reviews["developer"] == developer ]
    sentiment_count = developer_reviews["sentiment"].value_counts()

    return {developer: [{"Negative": sentiment_count.get("Negative", 0), "Positive": sentiment_count.get("Positive", 0)}]}

sentiment_analysis("Valve")

{'Valve': [{'Negative': 0, 'Positive': 0}]}

In [15]:
@app.get("/recommend/{title}")
def recommend_similar_games(title):

    # Encuentra el índice del juego ingresado
    game_index = model_data[model_data['title'] == title].index[0]

    # Filtra los juegos basados en la fecha de lanzamiento
    year = model_data["date"].dt.year[game_index]
    five_years_ago = year - 5
    five_years_later = year + 5
    filtered_data = model_data[(model_data["date"].dt.year >= five_years_ago) & (model_data["date"].dt.year <= five_years_later)]

    # Filtra los juegos basados price category
    price = model_data["price_discr"][game_index]
    upper_price = price + 1
    lower_price = price - 1
    filtered_data = filtered_data[(filtered_data["price_discr"] >= lower_price) & (filtered_data["price_discr"] <= upper_price)]
    
    filtered_data = filtered_data.reset_index(drop=True)
    new_game_index = filtered_data[filtered_data['title'] == title].index[0]

    # Create similarity matrix based on popular genres for filtered_data
    popular_genres_matrix = vectorizer.fit_transform(filtered_data['popular_genres'])
    popular_genres_similarity_matrix = cosine_similarity(popular_genres_matrix, popular_genres_matrix)
    # Create similarity matrix based on common genres for filtered_data
    common_genres_matrix = vectorizer.fit_transform(filtered_data['common_genres'])
    common_genres_similarity_matrix = cosine_similarity(common_genres_matrix, common_genres_matrix)
    # Create similarity matrix based on unpopular genres for filtered_data
    unpopular_genres_matrix = vectorizer.fit_transform(filtered_data['unpopular_genres'])
    unpopular_genres_similarity_matrix = cosine_similarity(unpopular_genres_matrix, unpopular_genres_matrix)
    # Combine the similarity matrices for the three genre categories
    similarity_matrix = popular_genres_similarity_matrix + common_genres_similarity_matrix + unpopular_genres_similarity_matrix

    # Adjust the game indices for the filtered_data
    similar_game_indices = similarity_matrix[new_game_index].argsort()[::-1][1:6]

    # Obtain the names of similar games and their similarity scores
    similar_game_scores = similarity_matrix[game_index][similar_game_indices]
    similar_game_titles = filtered_data.iloc[similar_game_indices]['title'].tolist()

    # Create a DataFrame with titles and similarity scores
    similar_games_dict = {'recommendations': similar_game_titles}

    return similar_games_dict

recommend_similar_games("Counter-Strike")

{'recommendations': ['Counter-Strike: Source',
  'Full Spectrum Warrior',
  "Tom Clancy's Ghost Recon® Desert Siege™",
  'Commandos: Beyond the Call of Duty',
  'Commandos: Behind Enemy Lines']}

In [ ]:
@app.get("/users_recommend/{year}") # Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
def UsersRecommend(year: int):

    year_reviews = australian_users_reviews[["item_id", "sentiment", "recommend"]][(pd.to_datetime(australian_users_reviews['date']).dt.year == year) & (australian_users_reviews['sentiment'] >= 1)]
    top_3_games = year_reviews["item_id"].value_counts().head(3)
    top_3_games = pd.DataFrame({'item_id': top_3_games.index, 'count': top_3_games.values})
    top_3_games = pd.merge(left=top_3_games, right=steam_games[['item_id', 'title']], on='item_id', how='left')
    most_recommended_games = [{"Puesto 1": top_3_games["title"][0]}, {"Puesto 2": top_3_games["title"][1]}, {"Puesto 3": top_3_games["title"][2]}]
    return most_recommended_games

UsersRecommend(2014)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Counter-Strike: Global Offensive'},
 {'Puesto 3': "Garry's Mod"}]

In [5]:
@app.get("/users_not_recommend/{year}") # Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
def UsersNotRecommend(year: int):
    year_reviews = australian_users_reviews[["item_id", "sentiment", "recommend"]][(pd.to_datetime(australian_users_reviews['date']).dt.year == year) & (australian_users_reviews['sentiment'] >= 1)]
    bot_3_games = year_reviews["item_id"].value_counts().tail(3)
    bot_3_games = pd.DataFrame({'item_id': bot_3_games.index, 'count': bot_3_games.values})
    bot_3_games = pd.merge(left=bot_3_games, right=steam_games[['item_id', 'title']], on='item_id', how='left')
    least_recommended_games = [{"Puesto 1": bot_3_games["title"][0]}, {"Puesto 2": bot_3_games["title"][1]}, {"Puesto 3": bot_3_games["title"][2]}]
    return least_recommended_games

UsersNotRecommend(2014)

NameError: name 'australian_users_reviews' is not defined